In [1]:
%matplotlib inline
import os
import sys
import glob
import logging
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import common.metrics as metrics
import numpy as np
import pandas as pd
import cPickle as pkl
import common.data_converter as dc
from common.plots import UnknownsDistributionPlot, RSquaredPlot
import lasagne_nn.output_loader as ol

/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 99, in init_dev
    **args)
  File "pygpu/gpuarray.pyx", line 658, in pygpu.gpuarray.init
  File 

In [2]:
# data handling

def get_env_var(handle):
    ''' Get an environment variable given the handle for the bash variable'''
    tmp = os.getenv(handle)
    if not tmp:
        raise LookupError("Environment variable: {} not set.".format(handle))
    return tmp.strip("'")

def get_preds(dset_dict, expt, dset, fold, ratio=None):
    indir = data_dict[expt]["indir"]
    if ratio is not None:
        knowns_file = data_dict[expt]["knowns_fmt"].format(indir, ratio, dset, fold)
        preds_file = data_dict[expt]["preds_fmt"].format(indir, ratio, dset, fold)
    else:
        knowns_file = data_dict[expt]["knowns_fmt"].format(indir, dset, fold)
        preds_file = data_dict[expt]["preds_fmt"].format(indir, dset, fold)
        
    prediction = np.load(preds_file)
    truth = np.load(knowns_file)
    
    if dset_dict[expt]["npz_fmt"] == True:
        prediction = prediction["arr_0"]
        truth = truth["arr_0"]
    return prediction, truth

In [3]:
def plot_rsquared(prediction, known_value, title="R Squared", img_filename="rsquared_{}.png", result_name='r2',
                  result_suffix='', color="blue", cmap_color="BuPu", fontsize=24, alpha=0.3):
    """
    Plot the r-squared between predicted and known values.
    Args:
        prediction (np.ndarray):
            value to be plotted on y-axis
        known_value (np.ndarray):
            value to be plotted on x-axis
        title (str):
            title of the plot
        img_filename (str):
            format to save as
        result_name (str):
            passed to compute_rsquared
    """
    figsize = (10, 10)
    sns.set(font_scale=4)
    sns.set_style("darkgrid")
    sns.set_context("poster")
    fig = plt.figure(figsize=figsize)

    t = known_value.ravel()
    nonzeros = ~np.isnan(t)
    nonzeros[nonzeros] &= t[nonzeros] > 0.0
    p = prediction.ravel()[nonzeros]
    t = t[nonzeros]
    output_dir = os.path.dirname(img_filename) if img_filename is not None else None
    logging.debug('plotting r^2')
    maxv = t.max() + 1
    g = sns.jointplot(x=p, y=t, alpha=alpha, xlim=(0, 14.5), ylim=(0, 14.5), stat_func=None, 
                      height=9, color=color, s=15, lw=0)
    rsquare = metrics.compute_rsquared(p, t, output_dir=output_dir, result_name=result_name,
                                       result_suffix=result_suffix)
    g = g.annotate((lambda x,y: rsquare), template="{stat}: {val:.4f}",
                   stat="$R^2$", loc="lower right", fontsize=fontsize)
    
    g.set_axis_labels("Predicted pAC50", "Known pAC50", fontsize=fontsize)
    g.ax_joint.tick_params(labelsize=fontsize)
    cmap = plt.cm.get_cmap(cmap_color)
    
    new_color_list = cmap(np.logspace(0.75, 1, 100))
    new_color_list[:, 3] = 0.95
    new_color_list[:-20, 3] = np.linspace(0.05, 0.9, 80)
    
    new_cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n='Al'+ cmap.name, a=0, b=100),
        new_color_list)
    
    g.ax_joint.hexbin(g.x, g.y, mincnt=2, cmap=new_cmap)
    g.fig.tight_layout()

    return g

In [4]:
def make_space_above(g, topmargin=1):
    """ increase figure size to make topmargin (in inches) space for 
        titles, without changing the axes sizes"""
    s = g.fig.subplotpars
    w, h = g.fig.get_size_inches()

    figh = h - (1-s.top)*h  + topmargin
    g.fig.subplots_adjust(bottom=s.bottom*h/figh, top=1-topmargin/figh)
    g.fig.set_figheight(figh)

In [5]:
reg_data_file = "./regression_preds_file_lookup.pkl"
with open(reg_data_file, "rb") as f:
    data_dict = pkl.load(f)

In [6]:
data_dict.keys()

['STD',
 'NEG_RM_SMA',
 'NEG_RM_scrambled',
 'scrambled_idx_LC',
 'NEG_RM_RATIOS_scrambled',
 'scrambled_idx',
 'NEG_UW',
 'STD_SMA_RATIOS',
 'NEG_RM',
 'NEG_RM_RATIOS',
 'SEA_SMA',
 'scrambled_idx_no_SMA',
 'STD_SMA',
 'scrambled_idx_no_SMA_LC']

In [7]:
with open("./color_dict_pt.pkl", "rb") as f: 
    color_dict_pt = pkl.load(f)
with open("./color_dict_bar.pkl", "rb") as f: 
    color_dict_bar = pkl.load(f)

In [8]:
save_dir = "{}/final_paper_figs".format(get_env_var("HOME"))

In [9]:
!mkdir -p $save_dir

In [10]:
title_fmter = "{}\n({}; Fold {})"
fontsize=28
title_fontsize=33

In [11]:
for fold in np.arange(0, 5, 1):
    for dset in ["drugmatrix", "timesplit", "test", "train"]:
        expt="STD"
        expt_rename="STD"
        ratio=None
        prediction, truth = get_preds(data_dict, expt, dset, fold, ratio=ratio)
        nan_mask = np.isnan(truth)
        g = plot_rsquared(prediction[~nan_mask], truth[~nan_mask], color="red", cmap_color="gist_heat_r", 
                          img_filename=None, fontsize=fontsize )
        g.fig.subplots_adjust(top=0.9)
        if dset == "drugmatrix":
            name="Drug Matrix"
        elif dset == "timesplit":
            name="Time Split"
        else:
            name = dset.capitalize()
        g.fig.suptitle(title_fmter.format(name, expt_rename, fold), y=1.0, fontsize=fontsize)
        make_space_above(g, topmargin=1.1) 
        plt.savefig("{}/{}_{}_{}_r-squared.png".format(save_dir, expt, dset, fold), dpi=300)
        plt.clf()
        print("n = {}".format(len(truth[~nan_mask])))

/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 3044


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


...

n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 93676


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 371872


In [12]:
# for fold in np.arange(0,5, 1):
#     for dset in ["test"]:
for fold in np.arange(0, 5, 1):
    for dset in ["drugmatrix", "timesplit", "test", "train"]:
        expt="STD_SMA"
        expt_rename = "SNA"
        ratio=None
        prediction, truth = get_preds(data_dict, expt, dset, fold, ratio=ratio)
        nan_mask = np.isnan(truth)
        g = plot_rsquared(prediction, truth, color="blue", cmap_color="BuPu", 
                          img_filename=None, fontsize=fontsize)
        g.fig.subplots_adjust(top=0.9)
        if dset == "drugmatrix":
            name="Drug Matrix"
        elif dset == "timesplit":
            name="Time Split"
        else:
            name = dset.capitalize()
        g.fig.suptitle(title_fmter.format(name, expt_rename, fold), y=1.0, fontsize=fontsize)
        make_space_above(g, topmargin=1.1)   
        plt.savefig("{}/{}_{}_{}_r-squared.png".format(save_dir, expt_rename, dset, fold), dpi=300)
        plt.clf()
        print("n = {}".format(len(truth[~nan_mask])))


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 3044


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


...

n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 93676


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 371872


In [13]:
for fold in np.arange(0,5,1):
    for dset in ["drugmatrix", "timesplit", "test", "train"]:
        expt="NEG_RM"
        expt_rename="Negatives Removed"
        ratio=None
        prediction, truth = get_preds(data_dict, expt, dset, fold, ratio=ratio)
        nan_mask = np.isnan(truth)
        g = plot_rsquared(prediction, truth, color="orange", cmap_color="YlOrBr", 
                          img_filename=None, fontsize=fontsize)
        g.fig.subplots_adjust(top=0.9)
        if dset == "drugmatrix":
            name="Drug Matrix"
        elif dset == "timesplit":
            name="Time Split"
        else:
            name = dset.capitalize()
        g.fig.suptitle(title_fmter.format(name, expt_rename, fold), y=1.0, fontsize=fontsize)
        make_space_above(g, topmargin=1.1)   
        plt.savefig("{}/{}_{}_{}_r-squared.png".format(save_dir, expt_rename, dset, fold), dpi=300)
        plt.clf()
        print("n = {}".format(len(truth[~nan_mask])))

/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 3044


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


...

n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 93676


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 371872


In [14]:
for fold in np.arange(0,5,1):
    for dset in ["drugmatrix", "timesplit", "test", "train"]:
        expt="NEG_RM_SMA"
        expt_rename="Negatives Removed +SNA"
        ratio="1.0"
        prediction, truth = get_preds(data_dict, expt, dset, fold, ratio=ratio)
        nan_mask = np.isnan(truth)
        g = plot_rsquared(prediction, truth, color="green", cmap_color="BuGn", 
                          img_filename=None, fontsize=fontsize)
        g.fig.subplots_adjust(top=0.9)
        if dset == "drugmatrix":
            name="Drug Matrix"
        elif dset == "timesplit":
            name="Time Split"
        else:
            name = dset.capitalize()
        g.fig.suptitle(title_fmter.format(name, expt_rename, fold), y=1.0, fontsize=fontsize)
        make_space_above(g, topmargin=1.1)   
        plt.savefig("{}/{}_{}_{}_r-squared.png".format(save_dir, expt_rename, dset, fold), dpi=300)
        plt.clf()
        print("n = {}".format(len(truth[~nan_mask])))

/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 3044


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


...

n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 93676


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 371872


In [15]:
# for fold in np.arange(0,5, 1):
#     for dset in ["test"]:
for fold in np.arange(0,5,1):
    for dset in ["drugmatrix", "timesplit", "test", "train"]:
        expt="scrambled_idx_no_SMA_LC"
        expt_rename="STD scrambled"
        ratio="1.0"
        prediction, truth = get_preds(data_dict, expt, dset, fold, ratio=ratio)
        nan_mask = np.isnan(truth)
        g = plot_rsquared(prediction, truth, color="gray", cmap_color="Greys", 
                          img_filename=None, fontsize=fontsize)
        g.fig.subplots_adjust(top=0.9)
        if dset == "drugmatrix":
            name="Drug Matrix"
        elif dset == "timesplit":
            name="Time Split"
        else:
            name = dset.capitalize()
        g.fig.suptitle(title_fmter.format(name, expt_rename, fold), y=1.0, fontsize=fontsize)
        make_space_above(g, topmargin=1.1)   
        plt.savefig("{}/{}_{}_{}_r-squared.png".format(save_dir, expt_rename, dset, fold), dpi=300)
        plt.clf()
        print("n = {}".format(len(truth[~nan_mask])))

/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 3044


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


...

n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 93676


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 371872


In [16]:
# for fold in np.arange(0,5, 1):
#     for dset in ["test"]:
for fold in np.arange(0,5,1):
    for dset in ["drugmatrix", "timesplit", "test", "train"]:
        expt="scrambled_idx_LC"
        expt_rename="SNA scrambled"
        ratio="1.0"
        prediction, truth = get_preds(data_dict, expt, dset, fold, ratio=ratio)
        nan_mask = np.isnan(truth)
        g = plot_rsquared(prediction, truth, color="purple", cmap_color="PuBu", 
                          img_filename=None, fontsize=fontsize)
        g.fig.subplots_adjust(top=0.9)
        if dset == "drugmatrix":
            name="Drug Matrix"
        elif dset == "timesplit":
            name="Time Split"
        else:
            name = dset.capitalize()
        g.fig.suptitle(title_fmter.format(name, expt_rename, fold), y=1.0, fontsize=fontsize)
        make_space_above(g, topmargin=1.1)   
        plt.savefig("{}/{}_{}_{}_r-squared.png".format(save_dir, expt_rename, dset, fold), dpi=300)
        plt.clf()
        print("n = {}".format(len(truth[~nan_mask])))

/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 3044


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


...

n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 93676


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 371872


In [17]:
for fold in np.arange(0,5,1):
    for dset in ["drugmatrix", "timesplit", "test", "train"]:
        expt="NEG_RM_RATIOS_scrambled"
        expt_rename="Negatives Removed +SNA scrambled"
        ratio="1.0"
        prediction, truth = get_preds(data_dict, expt, dset, fold, ratio=ratio)
        nan_mask = np.isnan(truth)
        g = plot_rsquared(prediction, truth, color="gold", cmap_color="YlOrBr", 
                          img_filename=None, fontsize=fontsize, alpha=0.9)
        g.fig.subplots_adjust(top=0.9)
        if dset == "drugmatrix":
            name="Drug Matrix"
        elif dset == "timesplit":
            name="Time Split"
        else:
            name = dset.capitalize()
        g.fig.suptitle(title_fmter.format(name, expt_rename, fold), y=1.0, fontsize=fontsize)
        make_space_above(g, topmargin=1.1)   
        plt.savefig("{}/{}_{}_{}_r-squared.png".format(save_dir, expt_rename, dset, fold), dpi=300)
        plt.clf()
        print("n = {}".format(len(truth[~nan_mask])))

/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 3044


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


...

n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 93676


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 371872


In [18]:
for fold in np.arange(0,5,1):
    for dset in ["drugmatrix", "timesplit", "test", "train"]:
        expt="NEG_RM_scrambled"
        expt_rename="Negatives Removed scrambled"
        ratio=None
        prediction, truth = get_preds(data_dict, expt, dset, fold, ratio=ratio)
        nan_mask = np.isnan(truth)
        g = plot_rsquared(prediction, truth, color="sienna", cmap_color="gist_heat_r", 
                          img_filename=None, fontsize=fontsize)
        g.fig.subplots_adjust(top=0.9)
        if dset == "drugmatrix":
            name="Drug Matrix"
        elif dset == "timesplit":
            name="Time Split"
        else:
            name = dset.capitalize()
        g.fig.suptitle(title_fmter.format(name, expt_rename, fold), y=1.0, fontsize=fontsize)
        make_space_above(g, topmargin=1.1)    
        plt.savefig("{}/{}_{}_{}_r-squared.png".format(save_dir, expt_rename, dset, fold), dpi=300)
        plt.clf()
        print("n = {}".format(len(truth[~nan_mask])))

/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 3044


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


...

n = 96863


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 93676


/srv/home/ecaceres/anaconda2/envs/features/lib/python2.7/site-packages/seaborn/axisgrid.py:1847: UserWarning: JointGrid annotation is deprecated and will be removed in a future release.
  warnings.warn(UserWarning(msg))


n = 371872
